In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_6/models/B5R2b5_AN_1FC_fold6_3.h5' #เปลี่ยน model ต้องเอา R2 ## 
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1_3.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1_3.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Traindf_fold6_3.csv') #เปลี่ยน Dataset
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(7344, 22)
(7344, 22)
Normal:  (3978, 22)
Abnormal:  (3366, 22)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,Path Crop,...,tagName,originalImage,left,top,width,height,Rleft,Rtop,Rwidth,Rheight
0,0,1,1,15,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,MildFattyLiver,https://irisprodseatraining.blob.core.windows....,141.0,96.0,586.0,614.0,0.143898,0.107041,0.595628,0.683267
1,1,11,11,15,P1,P1,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,ModerateFattyLiver,https://irisprodseatraining.blob.core.windows....,257.0,103.0,538.0,634.0,0.256617,0.115278,0.537205,0.705051
2,2,17,17,15,P1,P1,Abnormal,AB03,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,SevereFattyLiver,https://irisprodseatraining.blob.core.windows....,0.0,184.0,573.0,751.0,0.000862,0.190025,0.638344,0.773737
3,3,21,21,15,P1,P1,Abnormal,AB04,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,Cirrhosis,https://irisprodseatraining.blob.core.windows....,278.0,87.0,515.0,496.0,0.278395,0.097096,0.515427,0.551515
4,4,23,23,15,P1,P1,Abnormal,AB05,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,PDF1,https://irisprodseatraining.blob.core.windows....,249.0,69.0,439.0,385.0,0.254909,0.076894,0.448980,0.428283


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 7344 validated image filenames belonging to 2 classes.
{0: 'Abnormal', 1: 'Normal'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[-0.05791939, -0.1042993 ,  0.55999899, ..., -0.06254239,
         0.53255528, -0.09416784],
       [-0.20575137, -0.15928563,  0.18879609, ..., -0.08439566,
         0.40262771, -0.15352745],
       [-0.19491677, -0.12850678, -0.0252621 , ..., -0.11563949,
         0.33389404, -0.14296792],
       ...,
       [ 0.40354621,  0.51113635, -0.20246351, ...,  0.77638745,
        -0.19326736,  0.62263304],
       [ 0.02744253,  0.20907627, -0.19062436, ...,  0.58363551,
        -0.09143327,  0.41259497],
       [ 0.02782632,  0.27127823, -0.16562968, ...,  0.49477509,
        -0.17164442,  0.42118186]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.057919,-0.104299,0.559999,0.334676,-0.211875,0.143230,0.351069,-0.194544,-0.164327,0.162054,...,-0.154000,0.200516,-0.211840,-0.131218,-0.184839,0.000211,0.225336,-0.062542,0.532555,-0.094168
1,-0.205751,-0.159286,0.188796,0.215051,-0.205882,0.252646,0.195647,-0.230761,-0.191521,0.107694,...,-0.182562,0.113841,-0.220046,-0.168137,-0.201683,-0.163186,0.217547,-0.084396,0.402628,-0.153527
2,-0.194917,-0.128507,-0.025262,0.202529,-0.202233,0.146810,0.111522,-0.193777,-0.159751,0.086466,...,-0.182771,0.060222,-0.218887,-0.190204,-0.152746,-0.122986,0.055055,-0.115639,0.333894,-0.142968
3,-0.193768,-0.150722,-0.029017,0.215692,-0.219698,0.657656,0.575939,-0.219614,-0.125709,-0.034037,...,-0.165802,0.332176,-0.188523,-0.157068,-0.194065,0.411503,0.553589,-0.105359,0.675958,-0.175017
4,-0.165939,-0.173695,0.084442,0.479525,-0.223586,0.271616,0.371645,-0.211414,-0.205298,-0.062518,...,-0.140194,0.071116,-0.210337,-0.156080,-0.186756,0.678532,-0.021070,-0.096439,0.158125,-0.182546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7339,0.432871,0.512395,-0.203722,0.040187,0.578012,-0.019369,-0.175482,0.250025,0.197643,-0.191814,...,0.495447,-0.204198,0.758164,0.314160,0.590799,-0.148868,-0.174744,0.725114,-0.207958,0.394110
7340,0.235443,0.604847,-0.207820,0.054113,0.807931,0.138333,-0.176238,0.517748,0.240770,-0.038638,...,0.791391,-0.162944,0.779820,0.115583,0.725948,-0.154207,-0.180142,0.824987,-0.169584,0.573344
7341,0.403546,0.511136,-0.202464,-0.114024,0.715752,-0.066011,-0.172875,0.497135,0.253090,-0.092089,...,0.590053,-0.170830,1.042414,0.495610,0.794905,-0.160461,-0.174855,0.776387,-0.193267,0.622633
7342,0.027443,0.209076,-0.190624,-0.115367,0.366976,-0.155977,-0.190449,0.093199,0.041518,-0.133924,...,0.561786,-0.181887,0.626718,0.124115,0.373563,-0.177400,-0.195169,0.583636,-0.091433,0.412595


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,-0.057919,-0.104299,0.559999,0.334676,-0.211875,0.143230,0.351069,-0.194544,-0.164327,0.162054,...,-0.184839,0.000211,0.225336,-0.062542,0.532555,-0.094168,Abnormal,P1,FP-A,AB01
1,-0.205751,-0.159286,0.188796,0.215051,-0.205882,0.252646,0.195647,-0.230761,-0.191521,0.107694,...,-0.201683,-0.163186,0.217547,-0.084396,0.402628,-0.153527,Abnormal,P1,FP-A,AB02
2,-0.194917,-0.128507,-0.025262,0.202529,-0.202233,0.146810,0.111522,-0.193777,-0.159751,0.086466,...,-0.152746,-0.122986,0.055055,-0.115639,0.333894,-0.142968,Abnormal,P1,FP-A,AB03
3,-0.193768,-0.150722,-0.029017,0.215692,-0.219698,0.657656,0.575939,-0.219614,-0.125709,-0.034037,...,-0.194065,0.411503,0.553589,-0.105359,0.675958,-0.175017,Abnormal,P1,FP-A,AB04
4,-0.165939,-0.173695,0.084442,0.479525,-0.223586,0.271616,0.371645,-0.211414,-0.205298,-0.062518,...,-0.186756,0.678532,-0.021070,-0.096439,0.158125,-0.182546,Abnormal,P1,FP-A,AB05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7339,0.432871,0.512395,-0.203722,0.040187,0.578012,-0.019369,-0.175482,0.250025,0.197643,-0.191814,...,0.590799,-0.148868,-0.174744,0.725114,-0.207958,0.394110,Normal,P8,FP-E,Normal
7340,0.235443,0.604847,-0.207820,0.054113,0.807931,0.138333,-0.176238,0.517748,0.240770,-0.038638,...,0.725948,-0.154207,-0.180142,0.824987,-0.169584,0.573344,Normal,P8,FP-E,Normal
7341,0.403546,0.511136,-0.202464,-0.114024,0.715752,-0.066011,-0.172875,0.497135,0.253090,-0.092089,...,0.794905,-0.160461,-0.174855,0.776387,-0.193267,0.622633,Normal,P8,FP-E,Normal
7342,0.027443,0.209076,-0.190624,-0.115367,0.366976,-0.155977,-0.190449,0.093199,0.041518,-0.133924,...,0.373563,-0.177400,-0.195169,0.583636,-0.091433,0.412595,Normal,P8,FP-E,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVtrain_AN_RF_AN_fold6_3.csv') #เปลี่ยนชื่อไฟล์